In [ ]:
import os
import ast 
import math
import time
import torch
import logging
import smart_open

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gensim.utils import tokenize
from gensim.models.fasttext import FastText 

import warnings
warnings.simplefilter('ignore')

In [ ]:
from scripts.utils.hierarchy import *
from scripts.utils.processing import *
from scripts.utils.data_reading import *

logging.basicConfig(level=logging.INFO )

In [ ]:
o = OmniscienceReader("OmniScience/original/ArXiv_BMED_Evise_title_abstract_os.2018-07-11.tsv")

In [ ]:
temp = o.om_df

In [ ]:
temp.shape

In [ ]:
temp = temp.dropna()

In [ ]:
temp.shape

In [ ]:
temp["category"] = temp["file_id"].apply(lambda x: x.split(":")[0])

In [ ]:
grouped = temp.groupby('category')

In [ ]:
grouped.groups.keys()

In [ ]:
arxiv = grouped.get_group('ArXiv')
esvii = grouped.get_group('EVISE.PII')
bmed = grouped.get_group('BMED.PUI')

In [ ]:
print(esvii.shape)
print(arxiv.shape)
print(bmed.shape)

In [ ]:
esvii["omniscience_label_ids"] = esvii["omniscience_label_ids"].apply(lambda x: list(map(int, x[0])))

In [ ]:
def get_text(df):
    
    name = df.iloc[0]["category"]
    fname = "{}_raw.txt".format(name)
    
    file = open(fname, "wb+")

    for i in tqdm(df.index):
        str_each_line = df.at[i, "abstract"] + '\n'
        file.write(str_each_line.encode('utf-8'))

    file.close()
    
    return  name, fname

In [ ]:
class CorpusIter(object):
    def __init__(self, file_path):
        super(CorpusIter, self).__init__()
        self.file_path = file_path

    def __iter__(self):
        with smart_open.smart_open(self.file_path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))

In [ ]:
def fasttext_generator(fname, name):

    filename = "Omniscience/Fasttext/{}.model".format(name)

    if not os.path.isfile(filename):
        model = FastText(size=128, window=3, min_count=1) # hs=0, negative=0
        model.build_vocab(sentences=CorpusIter(fname))
        total_examples = model.corpus_count
        model.train(sentences=CorpusIter(fname), total_examples=total_examples, epochs=5)
        model.save(filename)
    else:
        model = FastText.load(filename)
    return model

In [ ]:
esvii["vec"] = 0

In [ ]:
e_name, e_fname = get_text(esvii)

In [ ]:
e_model = fasttext_generator(e_fname, e_name)

In [ ]:
for i, sent in enumerate(tqdm(CorpusIter(e_fname))):
    temp = 0.0
    slen = len(sent)
    for w in sent:
        temp += e_model.wv[w]
    temp /= slen
    esvii.iloc[i]["vec"] = temp